### Imports

In [35]:
import os
import pickle

from skimage import io
import pandas as pd
import cv2

from doctr.io import DocumentFile
from doctr.models import ocr_predictor

### Model Loading

In [7]:
model = ocr_predictor(pretrained=True)

/home/venkat/BADRI/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/venkat/BADRI/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


### DocTR Predictions on Docbank data with removed images

In [8]:
input_dir   = './docbank_processed/processed_data/ori_black/'
output_dir  = './docbank_processed/resultant_data/doctr/' 

In [9]:
dir_list = os.listdir(input_dir)
results = {}
images = []

for image in dir_list:
    doc = DocumentFile.from_images(input_dir+image)
    result = model(doc)
    results[image] = result
    images.append(image)
    
# for i in range(len(results)):
#     results[i].show(docs[i])

# with open(output_dir+'results.pkl', 'rb') as f:
#     results = pickle.load(f)

In [36]:
dimensions = {}
total = {}
for image,result in results.items():
    dim = tuple(reversed(result.pages[0].dimensions))
    predictions = []
    img = io.imread(os.path.join(input_dir,image))
    for block in result.pages[0].blocks:
        for line in block.lines:
            for word in line.words:
                values = []
                geo = word.geometry
                a = list(int(a*b) for a,b in zip(geo[0],dim))
                b = list(int(a*b) for a,b in zip(geo[1],dim))
                values.append(a[0])
                values.append(a[1])
                values.append(b[0])
                values.append(b[1])
                values.append(word.value)
                values.append(word.confidence)
                predictions.append(values)
                cv2.rectangle(img, (a[0], a[1]), (b[0], b[1]), (0, 0, 255))
    io.imsave(output_dir + 'images/' + image,img)
    total[image] = predictions

/tmp/ipykernel_1239653/3586465970.py:22: UserWarning: ./docbank_processed/resultant_data/doctr/images/148.tar_1707.02008.gz_ms_9_ori_pro.jpg is a low contrast image
  io.imsave(output_dir + 'images/' + image,img)


### Store Results

In [41]:
for image,result in total.items():
    name = image[:len(image) - 12]
    df = pd.DataFrame(result, columns = ['x0', 'y0', 'x1', 'y1', 'prediction', 'confidence'])
    df.to_csv(output_dir + 'txt/' + name + '.txt', sep='\t',index=False)

In [ ]:
with open(output_dir+'results.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(results, outp, pickle.HIGHEST_PROTOCOL)